# Pretraining & Transfer Learning using EfficientNet

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CZ4042 Project/EfficientNet

/content/drive/MyDrive/CZ4042 Project/EfficientNet


In [3]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00


In [4]:
from pathlib import Path
import multiprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import albumentations as A

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import applications
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.regularizers import l1_l2

from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization, GlobalAveragePooling2D
from efficientnet.keras import EfficientNetB0, EfficientNetB3


### Preprocessing IMDb-Wiki Data

In [5]:
# Data Augmentation
transforms = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.03125, scale_limit=0.20, rotate_limit=20, border_mode=cv2.BORDER_CONSTANT,value=0, p=1.0),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HorizontalFlip(p=0.5)
])

# Load images from directory and assign respective age and gender labels
class ImageSequence(Sequence):
    def __init__(self, df, mode):
        self.df = df
        self.indices = np.arange(len(df))
        self.batch_size = BATCH_SIZE
        self.img_dir = 'data/wiki_crop/'
        self.img_size = IMG_SIZE
        self.mode = mode

    def __getitem__(self, idx):
        sample_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        imgs = []
        genders = []
        ages = []

        for _, row in self.df.iloc[sample_indices].iterrows():
            img_path = self.img_dir + row["img_paths"]
            img = cv2.imread(str(img_path))
            if img is not None:
              img = cv2.resize(img, (self.img_size, self.img_size))

              if self.mode == "train":
                img = transforms(image=img)["image"]

              imgs.append(img)
              genders.append(row["genders"])
              ages.append(row["ages"])

        # Classify age into categories to match that of Adience
        age_buckets = []
        for age in ages:
            if int(age) in range(0, 2):
                age = 0
            elif int(age) in range(2, 6):
                age = 1
            elif int(age) in range(6, 13):
                age = 2
            elif int(age) in range(13, 20):
                age = 3
            elif int(age) in range(20, 32):
                age = 4
            elif int(age) in range(32, 43):
                age = 5
            elif int(age) in range(43, 60):
                age = 6
            elif int(age) >= 60:
                age = 7

            age_buckets.append(age)


        imgs = np.asarray(imgs,dtype=np.float32)
        genders = np.asarray(genders,dtype=np.float32)
        age_buckets = np.asarray(age_buckets,dtype=np.float32)

        return imgs, (genders, age_buckets)

    def __len__(self):
        return len(self.df) // self.batch_size

    def on_epoch_end(self):
        np.random.shuffle(self.indices)


In [6]:
# Parameters
BATCH_SIZE = 64
IMG_SIZE = 224
DROPOUT_RATE = 0.2
L_RATE = 0.001
EPOCHS = 5
RANDOM_STATE = 42
l1_reg=0.01
l2_reg=0.01

In [7]:
# Read serialized data labels and loading into Image Sequence for reading images
df = pd.read_csv("wiki.csv")
train, val = train_test_split(df, random_state=RANDOM_STATE, test_size=0.1)
train_gen = ImageSequence(train, "train")
val_gen = ImageSequence(val, "val")

### Model Architecture

In [10]:
# Initial EfficientNetB0 with no additional layers
def get_base_model(IMG_SIZE):
    base_model = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg")
    features = base_model.output
    pred_gender = Dense(units=2, activation="sigmoid", name="pred_gender")(features)
    pred_age = Dense(units=8, activation="softmax", name="pred_age")(features)
    model = Model(inputs=base_model.input, outputs=[pred_gender, pred_age])
    return model

In [11]:
# Function to get optimizer
def get_optimizer(optimizer_name, L_RATE):
    if optimizer_name == "sgd":
        return SGD(learning_rate=L_RATE, momentum=0.9, nesterov=True)
    elif optimizer_name == "adam":
        return Adam(learning_rate=L_RATE)
    else:
        raise ValueError("optimizer name should be 'sgd' or 'adam'")

# Learning rate scheduler
def get_scheduler(epochs, lrate):
    class Schedule:
        def __init__(self, nb_epochs, initial_lr):
            self.epochs = nb_epochs
            self.initial_lr = initial_lr

        def __call__(self, epoch_idx):
            if epoch_idx < self.epochs * 0.25:
                return self.initial_lr
            elif epoch_idx < self.epochs * 0.50:
                return self.initial_lr * 0.2
            elif epoch_idx < self.epochs * 0.75:
                return self.initial_lr * 0.04
            return self.initial_lr * 0.008
    return Schedule(epochs, lrate)

### Pre-training on Wiki Dataset

In [12]:
strategy = tf.distribute.MirroredStrategy()

# Instantiate and CompileModel with optimizer
with strategy.scope():
    model = get_base_model(IMG_SIZE)
    opt = get_optimizer("adam", L_RATE)
    scheduler = get_scheduler(EPOCHS, L_RATE)
    model.compile(optimizer=opt,
                      loss=["sparse_categorical_crossentropy", "sparse_categorical_crossentropy"],
                      metrics=['accuracy'])


16804768/16804768 [==============================] - 0s 0us/step


In [13]:
len(model.layers)

232

In [14]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 112, 112, 32)         128       ['stem_conv[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_activation (Activatio  (None, 112, 112, 32)         0         ['stem_bn[0][0]']         

In [12]:
# Save the pretrained model
filename = "_".join(["EfficientNetB0-Custom",
                         str(IMG_SIZE),
                         "weights.{epoch:02d}-{val_loss:.2f}.hdf5"])

callbacks=[]
callbacks.extend([LearningRateScheduler(schedule=scheduler), ModelCheckpoint(filename, monitor="val_loss", verbose=1, save_best_only=True,mode="auto")])

# Train model
model.fit(train_gen, epochs=EPOCHS, callbacks=callbacks, validation_data=val_gen,workers=multiprocessing.cpu_count())

Epoch 1/5
536/536 [==============================] - 2577s 5s/step - loss: 1.2278 - pred_gender_loss: 0.1801 - pred_age_loss: 1.0477 - pred_gender_accuracy: 0.9288 - pred_age_accuracy: 0.5717 - val_loss: 1.1765 - val_pred_gender_loss: 0.1353 - val_pred_age_loss: 1.0411 - val_pred_gender_accuracy: 0.9441 - val_pred_age_accuracy: 0.5885 - lr: 0.0010
Epoch 2/5
536/536 [==============================] - 92s 171ms/step - loss: 1.0270 - pred_gender_loss: 0.1141 - pred_age_loss: 0.9129 - pred_gender_accuracy: 0.9574 - pred_age_accuracy: 0.6249 - val_loss: 1.0313 - val_pred_gender_loss: 0.1256 - val_pred_age_loss: 0.9057 - val_pred_gender_accuracy: 0.9520 - val_pred_age_accuracy: 0.6245 - lr: 0.0010
Epoch 3/5
536/536 [==============================] - 90s 166ms/step - loss: 0.8556 - pred_gender_loss: 0.0721 - pred_age_loss: 0.7835 - pred_gender_accuracy: 0.9751 - pred_age_accuracy: 0.6748 - val_loss: 0.9161 - val_pred_gender_loss: 0.0742 - val_pred_age_loss: 0.8419 - val_pred_gender_accuracy: 

## Transfer Learning - Training on Adience Dataset

### Preprocessing

In [34]:
from matplotlib import image
from PIL import Image
from numpy import asarray

In [28]:
# Read data from folds
fold0 = pd.read_csv('data/fold_0_data.txt', sep='\t')
fold1 = pd.read_csv('data/fold_1_data.txt', sep='\t')
fold2 = pd.read_csv('data/fold_2_data.txt', sep='\t')
fold3 = pd.read_csv('data/fold_3_data.txt', sep='\t')
fold4 = pd.read_csv('data/fold_4_data.txt', sep='\t')
fold0

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,30601258@N03,10399646885_67c7d20df9_o.jpg,1,"(25, 32)",f,0,414,1086,1383,-115,30,17
1,30601258@N03,10424815813_e94629b1ec_o.jpg,2,"(25, 32)",m,301,105,640,641,0,0,94
2,30601258@N03,10437979845_5985be4b26_o.jpg,1,"(25, 32)",f,2395,876,771,771,175,-30,74
3,30601258@N03,10437979845_5985be4b26_o.jpg,3,"(25, 32)",m,752,1255,484,485,180,0,47
4,30601258@N03,11816644924_075c3d8d59_o.jpg,2,"(25, 32)",m,175,80,769,768,-75,0,34
...,...,...,...,...,...,...,...,...,...,...,...,...
4479,68094148@N04,11373794746_4720ac792a_o.jpg,478,"(25, 32)",m,664,0,242,211,-5,-15,73
4480,68094148@N04,11355711315_0f5b5da125_o.jpg,477,"(25, 32)",f,915,51,109,112,0,45,14
4481,10693681@N00,9162730346_b1bf71120a_o.jpg,479,"(25, 32)",m,2145,1270,249,249,10,30,25
4482,113830953@N04,11855529986_dff116e018_o.jpg,480,None,NaN,2878,1300,306,306,-100,0,164


In [29]:
# Read and classify age into categories
def fold_to_list(df):
    data2=[]
    for i in range(len(df)):
        path="data/aligned/"+str(df['user_id'][i])+"/landmark_aligned_face."+str(df['face_id'][i])+"."+str(df['original_image'][i])
        if df['gender'][i]=='m':
            gender=0
        else:
            gender=1

        if str(df['age'][i]) == str((0,2)):
            age=0
        elif str(df['age'][i]) == str((4,6)):
            age=1
        elif str(df['age'][i]) == str((8,13)):
            age=2
        elif str(df['age'][i]) == str((15,20)):
            age=3
        elif str(df['age'][i]) == str((25,32)):
            age=4
        elif str(df['age'][i]) == str((38,43)):
            age=5
        elif str(df['age'][i]) == str((48,53)):
            age=6
        elif str(df['age'][i]) == str((60,100)):
            age=7
        data2.append([path,gender,age])
    return data2

In [36]:
fold_list_0 = fold_to_list(fold0)
fold_list_1 = fold_to_list(fold1)
fold_list_2 = fold_to_list(fold2)
fold_list_3 = fold_to_list(fold3)
fold_list_4 = fold_to_list(fold4)

data_list = []

for i in (fold_list_0):
  data_list.append(i)
for i in (fold_list_1):
  data_list.append(i)
for i in (fold_list_2):
  data_list.append(i)
for i in (fold_list_3):
  data_list.append(i)
for i in (fold_list_4):
  data_list.append(i)

data_new = pd.DataFrame(data_list, columns = ['Path', 'Gender','Age'])
data_new.to_csv("data/data_adience_complied.csv",index=False)

data_new

,Path,Gender,Age
0,data/aligned/30601258@N03/landmark_aligned_fac...,1,4
1,data/aligned/30601258@N03/landmark_aligned_fac...,0,4
2,data/aligned/30601258@N03/landmark_aligned_fac...,1,4
3,data/aligned/30601258@N03/landmark_aligned_fac...,0,4
4,data/aligned/30601258@N03/landmark_aligned_fac...,0,4
...,...,...,...
19365,data/aligned/7153718@N04/landmark_aligned_face...,1,4
19366,data/aligned/7153718@N04/landmark_aligned_face...,1,4
19367,data/aligned/7153718@N04/landmark_aligned_face...,1,4
19368,data/aligned/7153718@N04/landmark_aligned_face...,1,4


In [ ]:
# Read image from directory according to path specified


data_image=[]
data_gender=[]
data_age=[]
for i in range(len(data_new)):
    img = cv2.imread(str(data_new['Path'][i]))
    print(i, data_new['Gender'][i], data_new['Age'][i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    data_image.append(img)
    data_gender.append(data_new['Gender'][i])
    data_age.append(data_new['Age'][i])

data_image = np.asarray(data_image,dtype=np.float32)
data_gender = np.asarray(data_gender,dtype=np.float32)
data_age = np.asarray(data_age,dtype=np.float32)

print(len(data_image))

In [103]:
# Convert to array for dataset
data_image = np.asarray(data_image,dtype=np.float32)
data_gender = np.asarray(data_gender,dtype=np.float32)
data_age = np.asarray(data_age,dtype=np.float32)

In [105]:
# Create train, test and validation splits
X_temp_image, X_test_image = train_test_split(data_image, test_size = 0.2, random_state = RANDOM_STATE)
y_temp_age, y_test_age = train_test_split(data_age, test_size = 0.2, random_state = RANDOM_STATE)
y_temp_gender, y_test_gender = train_test_split(data_gender, test_size = 0.2, random_state = RANDOM_STATE)

X_train_image, X_val_image = train_test_split(X_temp_image, test_size = 0.125, random_state = RANDOM_STATE)
y_train_age, y_val_age = train_test_split(y_temp_age, test_size = 0.125, random_state = RANDOM_STATE)
y_train_gender, y_val_gender = train_test_split(y_temp_gender, test_size = 0.125, random_state = RANDOM_STATE)

In [106]:
def adience_image_sequence (X_image, y_gender, y_age):
  return X_image, (y_gender, y_age)

class ImageSequenceAdience(Sequence):
  def __init__(self, X_image, y_gender, y_age):
    self.X = X_image
    self.y_age = y_age
    self.y_gender = y_gender
    self.df = data_new
    self.batch_size = BATCH_SIZE
    self.epochs = EPOCHS

  def __getitem__(self, idx):
    return self.X, (self.y_gender, self.y_age)

  def __len__(self):
      return len(self.df) // self.batch_size

  def on_epoch_end(self):
      return self.epochs

train_gen_adience = ImageSequenceAdience(X_train_image, y_train_gender, y_train_age)
val_gen_adience = ImageSequenceAdience(X_val_image, y_val_gender, y_val_age)
test_gen_adience = ImageSequenceAdience(X_test_image, y_test_gender, y_test_age)

### Training (Transfer Learning)

In [107]:
# Vanilla EfficientNetB0 with no additional layers
def get_base_model(IMG_SIZE):

    base_model = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling="avg")

    features = base_model.output
    pred_gender = Dense(units=2, activation="sigmoid", name="pred_gender")(features)
    pred_age = Dense(units=8, activation="softmax", name="pred_age")(features)
    model = Model(inputs=base_model.input, outputs=[pred_gender, pred_age])
    return model


In [ ]:
# Fine Tuned EfficientNetB5
def get_model(IMG_SIZE):

    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

    # Add custom layers on top of the pre-trained model
    features = base_model.output
    #features = BatchNormalization()(features)
    features = GlobalMaxPool2D(name='gap1')(features)
    #features = Dropout(DROPOUT_RATE, name='dropout1')(features)

    # Output Layer
    pred_gender = Dense(units=2, activation="sigmoid", name="pred_gender")(features)
    pred_age = Dense(units=8, activation="softmax", name="pred_age")(features)

    # Regularization for gender prediction
    pred_gender = Dense(units=2, activation="sigmoid", name="pred_gender",
                        kernel_regularizer=l1_l2(l1_reg, l2_reg))(base_model.output)

    # Regularization for age prediction
    pred_age = Dense(units=8, activation="softmax", name="pred_age",
                     kernel_regularizer=l1_l2(l1_reg, l2_reg))(base_model.output)

    model_final = Model(inputs=base_model.input, outputs=[pred_gender, pred_age])

    return model_final

In [15]:
# Path to pretrained model
weights_path = 'EfficientNetB0-Custom_224_weights.05-0.87.hdf5'

strategy = tf.distribute.MirroredStrategy()

# Instantiate model and Load it with pretrained weights. Freeze all layers so that it won't get updated again
with strategy.scope():
  pretrained_model = get_model(IMG_SIZE)
  pretrained_model.load_weights(weights_path)
  for layer in pretrained_model.layers:
    layer.trainable = False
  opt = get_optimizer("adam", L_RATE)
  scheduler = get_scheduler(EPOCHS, L_RATE)
  pretrained_model.compile(optimizer=opt,loss=["sparse_categorical_crossentropy", "sparse_categorical_crossentropy"],metrics=['accuracy'])


In [16]:
len(pretrained_model.layers)

232

In [ ]:
pretrained_model.fit(train_gen_adience, epochs=EPOCHS, callbacks=callbacks, validation_data=val_gen_adience,workers=multiprocessing.cpu_count())

In [ ]:
test_accuracy = pretrained_model.evaluate(test_gen_adience, verbose=1)
test_accuracy